In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import tensorflow as tf


In [ ]:
#this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in the Shared Google Drive
FOLDERNAME = 'Shared drives/CS 230 Project'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/{}'.format(FOLDERNAME))

%cd /content/drive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/Shared drives/CS 230 Project


In [ ]:
datapath = "datasets/Gutenberg/"
#train_df = pd.read_csv(datapath + "training100.csv").head(8000)
train_df = pd.read_csv(datapath + "training7.csv")
validation_df = pd.read_csv(datapath + "validation7.csv")
#validation_df = pd.read_csv(datapath + "validation100.csv")
#test_df = pd.read_csv(datapath + "test.csv")
#test_df = test_df.head(20000)

In [ ]:
len(train_df.index)

20000

In [ ]:
def map_authors(data):
    authors = []
    author_mappings = {}
    for index, row in data.iterrows():
        name = row['Author']
        if name in authors:
            continue
        authors.append(name)
        
    authors.sort()
    for i in range(len(authors)):
        author_mappings[authors[i]] = i
    return author_mappings 

In [ ]:
def reverse_mapping(author_mapping):
    reverse = {}
    for key in author_mapping.keys():
        reverse[author_mapping[key]] = key
    return reverse

In [ ]:

author_mapping = map_authors(validation_df)
print(author_mapping)
reverse_mappings = reverse_mapping(author_mapping)

{'Andrew Lang': 0, 'Anthony Trollope': 1, 'Bret Harte': 2, 'Charles Dickens': 3, 'Charles Kingsley': 4, 'Charlotte Mary Yonge': 5, 'Daniel Defoe': 6, 'Edgar Rice Burroughs': 7, 'Edward Phillips Oppenheim': 8, 'Edward Stratemeyer': 9, 'Frank Richard Stockton': 10, 'G K Chesterton': 11, 'George Alfred Henty': 12, 'George Bernard Shaw': 13, 'Hamlin Garland': 14, 'Harold Bindloss': 15, 'Henry James': 16, 'Henry Rider Haggard': 17, 'Herbert George Wells': 18, 'Jack London': 19, 'Jacob Abbott': 20, 'James Fenimore Cooper': 21, 'James Matthew Barrie': 22, 'James Otis': 23, 'Jerome Klapka Jerome': 24, 'John Galsworthy': 25, 'John Morley': 26, 'John Ruskin': 27, 'Joseph Conrad': 28, 'Louisa May Alcott': 29, 'Lyman Frank Baum': 30, 'Mark Twain': 31, 'Nathaniel Hawthorne': 32, 'Oscar Wilde': 33, 'P G Wodehouse': 34, 'R M Ballantyne': 35, 'Robert Louis Stevenson': 36, 'Rudyard Kipling': 37, 'Sir Arthur Conan Doyle': 38, 'Sir Walter Scott': 39, 'Thomas Carlyle': 40, 'Thomas Hardy': 41, 'Thomas Henr

In [ ]:
count_vectorizer = feature_extraction.text.CountVectorizer()
#train_vectors = count_vectorizer.fit_transform(train_df['Text'])
#test_vectors = count_vectorizer.fit_transform(test_df["Text"])

validation_vectors = count_vectorizer.fit_transform(validation_df["Text"])
train_vectors = count_vectorizer.transform(train_df['Text'])


ValueError: ignored

In [ ]:
def extract_features(data):
  features = np.zeros((len(data.index), 2))
  for index, row in data.iterrows():
      text = row['Text']
      features[index, 0] = len(text) / 100
      words = text.split()
      features[index, 1] = len(text) / len(words)
  return features


In [ ]:



def get_mappings(data, author_mapping):
    authors = data['Author']
    mappings = [author_mapping[author] for author in authors]
    data['mappings'] = mappings
    return data['mappings']

In [ ]:
#test_targets = get_mappings(test_df, author_mapping)
#test_targets = extract_features(test_df)
#train_vectors = extract_features(train_df)
#validation_vectors = extract_features(validation_df)
train_targets = get_mappings(train_df, author_mapping)
validation_targets = get_mappings(validation_df, author_mapping)

In [ ]:
train_targets

0        27
1        27
2        29
3        44
4        28
         ..
19728    20
19729    10
19730    49
19731    29
19732    30
Name: mappings, Length: 19733, dtype: int64

In [ ]:
train_df = train_df.rename(columns={'Text': 'text', 'mappings': 'Author_num'})
validation_df = validation_df.rename(columns={'Text': 'text', 'mappings': 'Author_num'})

In [ ]:

train_df[["text", "Author_num"]].head(10000).to_csv('Gutenberg_train_100_sentences.csv', index=False)

In [ ]:
validation_df[["text", "Author_num"]].head(2000).to_csv('Gutenberg_test_100_sentences.csv', index=False)